In [1]:
import os, sys, json

In [2]:

A4_DIR = "/home/anmola/assignments_hw/anlp_final_proj/outputs_A4"
# find all json files recursively in the directory
def find_json_files_in_dir(directory):
    json_files = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            # print(file)
            if file.endswith('.jsonl'):
                json_files.append(os.path.join(root, file))
    return json_files

json_files = find_json_files_in_dir(A4_DIR)

json_files = [x for x in json_files if "do_not_use" not in x]
json_files = [x for x in json_files if "DPO_NO" not in x]
print("Total number of json files: ", len(json_files))

# here is the filename: generated_codes_deepseek_base_1e-4_NO_cot_speed_incorrect_samples_10_epoch_edit_deepseek_instruct_nrowsNone_tokens1024_temp0.0_fewshotex0_samples1_2024-12-08_00:33:24.jsonl 

# remove timestamp from the filename BUT retain the extension
def remove_timestamp_from_filename(filename):
    return filename.rsplit('_', 2)[0] + ".jsonl"


# read all the json files and map the filename to the data
inference_outputs_df = {}
judge_results_df = {}

for file in json_files:
    print(f"{file=}")
    file_category = "infer" if "inference_outputs" in file else "judge"

    if file_category == "infer":
        with open(file, 'r') as f:
            data = f.readlines()
            data = [json.loads(x) for x in data]
            inference_outputs_df[remove_timestamp_from_filename(os.path.basename(file))] = data
    else:
        with open(file, 'r') as f:
            data = json.load(f)
            _use_filename = remove_timestamp_from_filename(os.path.basename(file))
            _use_filename = _use_filename.replace("generated_codes_1_", "")
            _use_filename = _use_filename.replace("generated_codes_", "")
            judge_results_df[_use_filename] = data

print("Total number of inference files: ", len(inference_outputs_df))
print("Total number of judge files: ", len(judge_results_df))
assert(len(inference_outputs_df) == len(judge_results_df))

# get sorted base filenames
sorted_base_filenames = sorted(list(inference_outputs_df.keys()) + list(judge_results_df.keys()))
print("Total number of sorted base filenames: ", len(sorted_base_filenames))
print("Example sorted base filenames: ", *sorted_base_filenames, sep='\n')
# number of unique elements print
print("Number of unique elements in sorted_base_filenames: ", len(set(sorted_base_filenames)))
unique_sorted_base_filenames = set(sorted_base_filenames)


Total number of json files:  18
file='/home/anmola/assignments_hw/anlp_final_proj/outputs_A4/DPO_SPEED_AND_PASS1_FILTERING/judge_outputs/generated_codes_deepseek_base_1e-4_NO_cot_speed_incorrect_samples_10_epoch_edit_deepseek_instruct_nrowsNone_tokens1024_temp0.0_fewshotex0_samples1_2024-12-08_00:33:24.jsonl'
file='/home/anmola/assignments_hw/anlp_final_proj/outputs_A4/DPO_SPEED_AND_PASS1_FILTERING/judge_outputs/generated_codes_deepseek_base_1e-4_NO_cot_speed_incorrect_samples_10_epoch_edit_deepseek_instruct_nrowsNone_tokens1024_temp0.0_fewshotex2_samples1_2024-12-07_22:43:54.jsonl'
file='/home/anmola/assignments_hw/anlp_final_proj/outputs_A4/DPO_SPEED_AND_PASS1_FILTERING/judge_outputs/generated_codes_1_deepseek_base_1e-4_NO_cot_speed_incorrect_samples_10_epoch_self-refine_deepseek_instruct_nrowsNone_tokens1024_temp0.0_fewshotex0_samples1_2024-12-07_23:21:55.jsonl'
file='/home/anmola/assignments_hw/anlp_final_proj/outputs_A4/DPO_SPEED_AND_PASS1_FILTERING/inference_outputs/deepseek_base

In [3]:
assert(len(unique_sorted_base_filenames) == len(inference_outputs_df))


mapping_df = {x: "" for x in unique_sorted_base_filenames }

# with open("mapping.json", 'r') as f:
#     mapping_df = json.load(f)

for key in mapping_df:
    key_ans = []
    
    #######################
    list1 = ["fewshotex0", "fewshotex2"]
    # make sure atleast one is present
    assert(any(x in key for x in list1))
    if "fewshotex0" in key:
        key_ans.append("zeroshot")
    else:
        key_ans.append("fewshot")
    #############
    list2 = ["only_failed", "only_speed", "speed_incorrect"]
    # make sure atleast one is present
    assert(any(x in key for x in list2))
    if "only_failed" in key:
        key_ans.append("only-failed")
    elif "only_speed" in key:
        key_ans.append("only-speed")
    else:
        key_ans.append("speed-incorrect")
    ################
    list3 = ["edit", "self-refine"]
    # make sure atleast one is present
    assert(any(x in key for x in list3))
    if "edit" in key:
        key_ans.append("edit")
    else:
        key_ans.append("self-refine")
    #####
    final_key = "_".join(key_ans)
    mapping_df[key] = final_key


In [4]:
# make sure all unique_sorted_base_filenames are in mapping_df, raise error if not
for x in unique_sorted_base_filenames:
    if x not in mapping_df:
        print(f"Error: {x} not in mapping_df")
        raise ValueError

# replace the keys in inference_outputs_df and judge_results_df with the new keys
inference_outputs_df = {mapping_df[k]: v for k, v in inference_outputs_df.items()}
judge_results_df = {mapping_df[k]: v for k, v in judge_results_df.items()}

all_keys = set(inference_outputs_df.keys()) | set(judge_results_df.keys())
assert(all_keys == set(inference_outputs_df.keys()) == set(judge_results_df.keys()))
#######################
#### NOW START ASSEMBLING STUFF FOR ANALYSIS
# read each print keys in first element
for key_now in all_keys:
    print(f"{key_now=}")
    print(f"{inference_outputs_df[key_now][0].keys()}")
    print(f"{judge_results_df[key_now]['0'].keys()}")
    print("\n########")

key_now='zeroshot_only-speed_self-refine'
dict_keys(['input', 'target', 'problem_id', 'prompt', 'feedback_prompt', 'refine_prompt', 'feedback_0', 'generated_codes_0', 'full_generations_0', 'generated_codes_1', 'full_generations_1'])
dict_keys(['problem_id', 'input_accepted', 'output_accepted', 'input_run_time', 'best_run_time', 'input_memory', 'best_memory', 'speedup', 'mem_reduction', 'input_pass_rate', 'output_pass_rate', 'input_run_time_all', 'input_memory_all', 'input_pass_all', 'output_run_time_all', 'output_memory_all', 'output_pass_all', 'input_errors_all', 'output_errors_all'])

########
key_now='fewshot_only-failed_edit'
dict_keys(['input', 'target', 'problem_id', 'generated_codes', 'full_generations', 'prompt'])
dict_keys(['problem_id', 'input_accepted', 'output_accepted', 'input_run_time', 'best_run_time', 'input_memory', 'best_memory', 'speedup', 'mem_reduction', 'input_pass_rate', 'output_pass_rate', 'input_run_time_all', 'input_memory_all', 'input_pass_all', 'output_run_t

In [5]:
class ModelCode:
    problem_id: str
    instance_id: int
    code_str: str
    passed_testcases_list: list
    error_trace: dict
    tc_list: list # has status ie passed or failed, time taken, error message, memory taken
    setting_id: str

In [6]:
def prep_output_object(code_str, problem_id, instance_id, setting_id, pass_list, runtime_map, memory_map, error_map):
    model_code = ModelCode()
    model_code.code_str = code_str
    model_code.problem_id = problem_id
    model_code.instance_id = instance_id
    model_code.setting_id = setting_id
    model_code.passed_testcases_list = sorted(pass_list) # list of ints
    failed_cases = [int(x) for x in error_map.keys()]
    # print("Number of passed testcases: ", len(model_code.passed_testcases_list))
    # print("Number of failed testcases: ", len(failed_cases))
    # print("Number of keys with runtime: ", len(runtime_map))
    # print("Number of keys with memory: ", len(memory_map))
    # make sure failed cases and passed cases are non-overlapping
    assert(len(set(failed_cases) & set(model_code.passed_testcases_list)) == 0)
    all_tests = set(failed_cases) | set(model_code.passed_testcases_list)
    all_tests = sorted(list(all_tests))
    # assert(all_tests == list(range(0, len(all_tests))))
    # keys in maps are string version of integers
    model_code.tc_list = []
    for tc_id in all_tests:
        if str(tc_id) in error_map:
            # assert(str(tc_id) not in runtime_map)
            # assert(str(tc_id) not in memory_map)
            model_code.tc_list.append({
                "tc_id": tc_id,
                "status": "failed",
                "time_taken": runtime_map.get(str(tc_id), None),
                "error_message": error_map[str(tc_id)],
                "memory_taken": memory_map.get(str(tc_id), None)    
            })
        else:
            assert(str(tc_id) in runtime_map)
            assert(str(tc_id) in memory_map)
            model_code.tc_list.append({
                "tc_id": tc_id,
                "status": "passed",
                "time_taken": runtime_map[str(tc_id)],
                "error_message": "",
                "memory_taken": memory_map[str(tc_id)]
            })
    return model_code

In [7]:
instance_df = dict()

In [8]:
for setting_id in all_keys:
    print(f"{setting_id}")
    judge_elems = list(judge_results_df[setting_id].values())
    infer_elems = inference_outputs_df[setting_id]
    assert(len(judge_elems) == len(infer_elems))
    for instance_id, (infer_elem, judge_elem) in enumerate(zip(infer_elems, judge_elems)):
        assert(infer_elem['problem_id'] == judge_elem['problem_id'])
        code_prep_key = ["generated_codes", "generated_codes_0", "generated_codes_1", "generated_codes_2"]
        # find key which is latest in code_prep_key and still occurs in infer_elem
        code_str = None
        for key in code_prep_key:
            if key in infer_elem:
                code_str =key
                # break
        # print(f"{code_str=}")
        # output details
        infer_code_str = infer_elem[code_str][0]
        infer_problem_id = infer_elem['problem_id']
        infer_setting_id = "output_"+setting_id
        infer_instance_id = instance_id
        infer_pass_list = judge_elem['output_pass_all']
        infer_runtime_map = judge_elem['output_run_time_all']
        infer_memory_map = judge_elem['output_memory_all']
        infer_error_map = judge_elem['output_errors_all']
        infer_model_code = prep_output_object(infer_code_str, infer_problem_id, infer_instance_id, infer_setting_id, infer_pass_list, infer_runtime_map, infer_memory_map, infer_error_map)

        # input details
        input_code_str = infer_elem["input"]
        input_problem_id = infer_elem['problem_id']
        input_setting_id = "input_"+setting_id
        input_instance_id = instance_id
        input_pass_list = judge_elem['input_pass_all']
        input_runtime_map = judge_elem['input_run_time_all']
        input_memory_map = judge_elem['input_memory_all']
        input_error_map = judge_elem['input_errors_all']
        input_model_code = prep_output_object(input_code_str, input_problem_id, input_instance_id, input_setting_id, input_pass_list, input_runtime_map, input_memory_map, input_error_map)
        if instance_id not in instance_df:
            instance_df[instance_id] = dict()
        instance_df[instance_id][input_setting_id] = input_model_code
        instance_df[instance_id][infer_setting_id] = infer_model_code

        
        
        # break
    # break

zeroshot_only-speed_self-refine
fewshot_only-failed_edit
fewshot_speed-incorrect_edit
zeroshot_speed-incorrect_self-refine
zeroshot_speed-incorrect_edit
zeroshot_only-failed_edit
zeroshot_only-failed_self-refine
zeroshot_only-speed_edit
fewshot_only-speed_edit


In [9]:
instance_df

{0: {'input_zeroshot_only-speed_self-refine': <__main__.ModelCode at 0x7fd99c2f9610>,
  'output_zeroshot_only-speed_self-refine': <__main__.ModelCode at 0x7fd99c2f9040>,
  'input_fewshot_only-failed_edit': <__main__.ModelCode at 0x7fd97c78a070>,
  'output_fewshot_only-failed_edit': <__main__.ModelCode at 0x7fd97c78a040>,
  'input_fewshot_speed-incorrect_edit': <__main__.ModelCode at 0x7fd973f4beb0>,
  'output_fewshot_speed-incorrect_edit': <__main__.ModelCode at 0x7fd973f4be80>,
  'input_zeroshot_speed-incorrect_self-refine': <__main__.ModelCode at 0x7fd9736d9d00>,
  'output_zeroshot_speed-incorrect_self-refine': <__main__.ModelCode at 0x7fd9736d9cd0>,
  'input_zeroshot_speed-incorrect_edit': <__main__.ModelCode at 0x7fd972e56ca0>,
  'output_zeroshot_speed-incorrect_edit': <__main__.ModelCode at 0x7fd972e566d0>,
  'input_zeroshot_only-failed_edit': <__main__.ModelCode at 0x7fd9725e2a00>,
  'output_zeroshot_only-failed_edit': <__main__.ModelCode at 0x7fd9725e29d0>,
  'input_zeroshot_onl

#### Make sure that for same problem, for input based settings, set of passed and failed test cases is same

In [10]:
for instance_id in instance_df:
    print(f"{instance_id=}")
    print(f"{instance_df[instance_id].keys()}")
    print("\n########")
    all_input_settings = [x for x in instance_df[instance_id].keys() if "input" in x]
    passed_list = None
    all_tests  = None
    for setting_id in all_input_settings:
        setting_passed_list = instance_df[instance_id][setting_id].passed_testcases_list
        setting_all_tests = [x['tc_id'] for x in instance_df[instance_id][setting_id].tc_list]
        if passed_list is None:
            passed_list = set(setting_passed_list)
            all_tests = set(setting_all_tests)
        else:
            # make sure equal
            assert(passed_list == set(setting_passed_list))
            assert(all_tests == set(setting_all_tests))

instance_id=0
dict_keys(['input_zeroshot_only-speed_self-refine', 'output_zeroshot_only-speed_self-refine', 'input_fewshot_only-failed_edit', 'output_fewshot_only-failed_edit', 'input_fewshot_speed-incorrect_edit', 'output_fewshot_speed-incorrect_edit', 'input_zeroshot_speed-incorrect_self-refine', 'output_zeroshot_speed-incorrect_self-refine', 'input_zeroshot_speed-incorrect_edit', 'output_zeroshot_speed-incorrect_edit', 'input_zeroshot_only-failed_edit', 'output_zeroshot_only-failed_edit', 'input_zeroshot_only-failed_self-refine', 'output_zeroshot_only-failed_self-refine', 'input_zeroshot_only-speed_edit', 'output_zeroshot_only-speed_edit', 'input_fewshot_only-speed_edit', 'output_fewshot_only-speed_edit'])

########
instance_id=1
dict_keys(['input_zeroshot_only-speed_self-refine', 'output_zeroshot_only-speed_self-refine', 'input_fewshot_only-failed_edit', 'output_fewshot_only-failed_edit', 'input_fewshot_speed-incorrect_edit', 'output_fewshot_speed-incorrect_edit', 'input_zeroshot_s